# Load Data to SQL Server

This notebook loads the transformed data from the Silver layer into SQL Server using a dimensional model with dimension and fact tables.

## Process Overview
1. Set up environment and initialize Spark session
2. Read data from the Silver layer
3. Create dimension tables (DimDate, DimDepartment, DimProductCategory, DimProduct)
4. Create fact table (FactProduction)
5. Load all tables to SQL Server

## 1. Environment Setup

Initialize Spark session and import required libraries.

In [ ]:
# Set JAVA_HOME environment variable
import os
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jre1.8.0_451'

# Import required libraries
from datetime import datetime
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyodbc
import logging
import glob

# Configure logging
date_str = datetime.now().strftime("%Y-%m-%d")
log_dir = os.path.join("../logs", "sql_load", date_str)
os.makedirs(log_dir, exist_ok=True)
log_path = os.path.join(log_dir, "sql_load.log")

logging.basicConfig(filename=log_path, level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def log_message(message, level="info"):
    """Logs messages with the specified level."""
    if level == "info":
        logging.info(message)
    elif level == "error":
        logging.error(message)
    print(message)

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Load to SQL Server") \
    .config("spark.driver.extraClassPath", "C:/spark-3.4.3/sqljdbc_4.2.8112.200_enu/sqljdbc_4.2/enu/jre8/sqljdbc42.jar") \
    .getOrCreate()

log_message("Environment setup completed")

## 2. Path Configuration and Data Loading

Define paths and load data from the Silver layer.

In [ ]:
# Define paths
current_dir = os.path.dirname(os.path.abspath("__file__"))
base_dir = os.path.dirname(current_dir)  # Go up one level to reach the root directory
silver_base_path = os.path.join(base_dir, "output", "silverLayer")

# Find the latest date folder in silver layer
silver_date_folders = glob.glob(os.path.join(silver_base_path, "*"))
if not silver_date_folders:
    raise Exception(f"No date folders found in {silver_base_path}")

latest_silver_folder = max(silver_date_folders)
silver_path = latest_silver_folder

print(f"Silver path: {silver_path}")

# Read the silver layer data
try:
    # Read data from silver layer
    silver_df = spark.read.parquet(os.path.join(silver_path, "mrp_production.parquet"))
    log_message(f"Successfully read data from {os.path.join(silver_path, 'mrp_production.parquet')}")
    print(f"Row count: {silver_df.count()}")
    print(f"Column count: {len(silver_df.columns)}")
    
    # Display column names
    print("\nColumn names in the dataset:")
    for col_name in silver_df.columns:
        print(f"- {col_name}")
    
    # Convert to pandas for easier processing
    df = silver_df.toPandas()
    log_message("Converted to pandas DataFrame")
    
    # Display sample data
    display(df.head())
except Exception as e:
    log_message(f"Error reading silver layer data: {str(e)}", level="error")
    import traceback
    log_message(traceback.format_exc(), level="error")

## 3. SQL Server Connection Setup

Configure connection to SQL Server and define helper functions.

In [ ]:
# SQL Server connection parameters
server = 'localhost'
database = 'DW_ODOO'
trusted_connection = 'yes'  # Windows authentication

# Create connection string
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection={trusted_connection}'

# Function to execute SQL query
def execute_sql(query, params=None):
    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        conn.commit()
        cursor.close()
        conn.close()
        return True
    except Exception as e:
        log_message(f"Error executing SQL: {str(e)}", level="error")
        return False

# Function to load data to SQL Server
def load_dataframe_to_sql(df, table_name, if_exists='append'):
    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        
        # Generate insert statements
        for index, row in df.iterrows():
            columns = ', '.join(df.columns)
            placeholders = ', '.join(['?' for _ in range(len(df.columns))])
            query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
            
            # Convert any NaN values to None
            values = [None if pd.isna(val) else val for val in row]
            
            cursor.execute(query, values)
            
            # Commit every 1000 rows to avoid memory issues
            if index % 1000 == 0:
                conn.commit()
                print(f"Committed {index} rows to {table_name}")
        
        conn.commit()
        cursor.close()
        conn.close()
        log_message(f"Successfully loaded {len(df)} rows to {table_name}")
        return True
    except Exception as e:
        log_message(f"Error loading data to {table_name}: {str(e)}", level="error")
        return False

# Test connection
try:
    conn = pyodbc.connect(conn_str)
    print("Successfully connected to SQL Server")
    conn.close()
except Exception as e:
    print(f"Error connecting to SQL Server: {str(e)}")

## 4. Create and Load Dimension Tables

Process the data and create dimension tables.

### 4.1 Create DimDate Table

In [ ]:
# Create DimDate
log_message("Creating DimDate table...")

# Get all dates from the dataset (Start, End, Deadline)
all_dates = pd.concat([
    df['Start'].dropna(),
    df['End'].dropna(),
    df['Deadline'].dropna()
]).drop_duplicates().sort_values().reset_index(drop=True)

# Create date dimension
dim_date = pd.DataFrame({
    'Date': all_dates,
    'Year': all_dates.dt.year,
    'Quarter': all_dates.dt.quarter,
    'Month': all_dates.dt.month,
    'MonthName': all_dates.dt.strftime('%B'),
    'Day': all_dates.dt.day,
    'DayOfWeek': all_dates.dt.dayofweek + 1,  # 1-7 instead of 0-6
    'DayOfWeekName': all_dates.dt.strftime('%A')
})

# Create DateKey (YYYYMMDD format)
dim_date['DateKey'] = dim_date['Date'].dt.strftime('%Y%m%d').astype(int)

# Reorder columns
dim_date = dim_date[['DateKey', 'Date', 'Year', 'Quarter', 'Month', 'MonthName', 'Day', 'DayOfWeek', 'DayOfWeekName']]

# Display sample data
display(dim_date.head())
print(f"Total dates: {len(dim_date)}")

# Load DimDate to SQL
# load_dataframe_to_sql(dim_date, 'dbo.DimDate', if_exists='replace')

### 4.2 Create DimDepartment Table

In [ ]:
# Create DimDepartment
log_message("Creating DimDepartment table...")

# Get unique departments
departments = df['Responsible'].dropna().unique()
dim_department = pd.DataFrame({
    'DepartmentID': [f"DEPT_{i+1}" for i in range(len(departments))],
    'DepartmentName': departments
})

# Display the department dimension
display(dim_department)

# Create a mapping of department names to keys
department_mapping = {}
for i, dept in enumerate(departments):
    department_mapping[dept] = i + 1  # DepartmentKey starts at 1

# Load DimDepartment to SQL
# load_dataframe_to_sql(dim_department, 'dbo.DimDepartment', if_exists='replace')

### 4.3 Create DimProductCategory Table

In [ ]:
# Create DimProductCategory
log_message("Creating DimProductCategory table...")

# Get unique product categories
product_categories = df['Product_Category'].dropna().unique()
dim_product_category = pd.DataFrame({
    'ProductCategoryName': product_categories
})

# Display the product category dimension
display(dim_product_category)

# Create a mapping of category names to keys
category_mapping = {}
for i, cat in enumerate(product_categories):
    category_mapping[cat] = i + 1  # ProductCategoryKey starts at 1

# Load DimProductCategory to SQL
# load_dataframe_to_sql(dim_product_category, 'dbo.DimProductCategory', if_exists='replace')

### 4.4 Create DimProduct Table

In [ ]:
# Create DimProduct
log_message("Creating DimProduct table...")

# Get unique products
products = df[['Product_Code', 'Product_Name', 'Product_Category', 'Product_Cost', 
              'Product_Sales_Price', 'Profit_Margin_Percent', 'Price_Category', 'Margin_Category']].drop_duplicates()

# Add ProductCategoryKey
products['ProductCategoryKey'] = products['Product_Category'].map(category_mapping)

# Rename columns to match SQL table
dim_product = products.rename(columns={
    'Product_Code': 'ProductCode',
    'Product_Name': 'ProductName',
    'Product_Cost': 'ProductCost',
    'Product_Sales_Price': 'ProductSalesPrice',
    'Profit_Margin_Percent': 'ProfitMarginPercent',
    'Price_Category': 'PriceCategory',
    'Margin_Category': 'MarginCategory'
})

# Select only the columns we need
dim_product = dim_product[['ProductCode', 'ProductName', 'ProductCategoryKey', 'ProductCost', 
                          'ProductSalesPrice', 'ProfitMarginPercent', 'PriceCategory', 'MarginCategory']]

# Display the product dimension
display(dim_product.head())
print(f"Total products: {len(dim_product)}")

# Create a mapping of product codes to keys
product_mapping = {}
for i, code in enumerate(products['Product_Code'].unique()):
    product_mapping[code] = i + 1  # ProductKey starts at 1

# Load DimProduct to SQL
# load_dataframe_to_sql(dim_product, 'dbo.DimProduct', if_exists='replace')

## 5. Create and Load Fact Table

Process the data and create the fact table.

In [ ]:
# Create FactProduction
log_message("Creating FactProduction table...")

# Create date key mapping function
def date_to_key(date):
    if pd.isna(date):
        return None
    return int(date.strftime('%Y%m%d'))

# Create the fact table
fact_production = df[['Reference', 'Product_Code', 'Responsible', 'Start', 'End', 'Deadline',
                     'State', 'Quantity_Producing', 'Quantity_To_Produce', 'Total_Quantity',
                     'Production_Efficiency', 'Production_Duration_Days']].copy()

# Map dimension keys
fact_production['ProductKey'] = fact_production['Product_Code'].map(product_mapping)
fact_production['DepartmentKey'] = fact_production['Responsible'].map(department_mapping)
fact_production['StartDateKey'] = fact_production['Start'].apply(date_to_key)
fact_production['EndDateKey'] = fact_production['End'].apply(date_to_key)
fact_production['DeadlineDateKey'] = fact_production['Deadline'].apply(date_to_key)

# Drop original columns that have been mapped
fact_production = fact_production.drop(['Product_Code', 'Responsible', 'Start', 'End', 'Deadline'], axis=1)

# Rename remaining columns
fact_production = fact_production.rename(columns={
    'Quantity_Producing': 'QuantityProducing',
    'Quantity_To_Produce': 'QuantityToProduce',
    'Total_Quantity': 'TotalQuantity',
    'Production_Efficiency': 'ProductionEfficiency',
    'Production_Duration_Days': 'ProductionDurationDays'
})

# Display the fact table
display(fact_production.head())
print(f"Total fact records: {len(fact_production)}")

# Load FactProduction to SQL
# load_dataframe_to_sql(fact_production, 'dbo.FactProduction', if_exists='replace')

## 6. Execute the Data Loading

Uncomment the loading commands and execute them to load the data to SQL Server.

In [ ]:
# Load all tables to SQL Server
try:
    # Load dimension tables
    print("Loading DimDate...")
    load_dataframe_to_sql(dim_date, 'dbo.DimDate', if_exists='replace')
    
    print("Loading DimDepartment...")
    load_dataframe_to_sql(dim_department, 'dbo.DimDepartment', if_exists='replace')
    
    print("Loading DimProductCategory...")
    load_dataframe_to_sql(dim_product_category, 'dbo.DimProductCategory', if_exists='replace')
    
    print("Loading DimProduct...")
    load_dataframe_to_sql(dim_product, 'dbo.DimProduct', if_exists='replace')
    
    # Load fact table
    print("Loading FactProduction...")
    load_dataframe_to_sql(fact_production, 'dbo.FactProduction', if_exists='replace')
    
    log_message("All tables loaded successfully to SQL Server!")
except Exception as e:
    log_message(f"Error loading tables to SQL Server: {str(e)}", level="error")
    import traceback
    log_message(traceback.format_exc(), level="error")

## 7. Cleanup

Stop the Spark session and clean up resources.

In [ ]:
# Stop Spark session
spark.stop()
log_message("Spark session stopped.")
log_message("Data loading process completed.")